In [54]:
import pandas as pd
import numpy as np

from dash_website.utils.aws_loader import load_feather
from dash_website.utils.controls import get_drop_down, get_item_radio_items, get_options
from dash_website.utils.graphs import add_line_and_annotation
from dash_website import DOWNLOAD_CONFIG, CUSTOM_ORDER, ALGORITHMS_RENDERING
from dash_website.age_prediction_performances import SAMPLE_DEFINITION, DIMENSIONS_SELECTION, SCORES, CUSTOM_DIMENSIONS

metric = "r2"
scores =  load_feather(f"age_prediction_performances/scores_all_samples_per_participant.feather").set_index(["dimension", "subdimension", "sub_subdimension"])

scores = scores.loc[[("BloodCells", "BloodCount", "Scalars"), ("Biochemistry", "Blood", "Scalars"), ("Biochemistry", "Urine", "Scalars")]]

In [62]:
import plotly.graph_objs as go

sorted_dimensions = scores.index.drop_duplicates()


x_positions = pd.DataFrame(
    np.arange(5, 10 * len(sorted_dimensions) + 5, 10), index=sorted_dimensions, columns=["x_position"]
)

fig = go.Figure()
fig.update_layout(
    xaxis={
        "tickvals": np.arange(5, 10 * len(sorted_dimensions) + 5, 10),
        "ticktext": ["Blood Cells", "Blood Biochemistry", "Urine Biochemistry"],
    }
)

algorithms = scores["algorithm"].drop_duplicates()

hovertemplate = (
    "%{x}, score: %{y:.3f} +- %{customdata[0]:.3f}, sample size: %{customdata[1]} <extra>%{customdata[2]}</extra>"
)

min_score = min(scores[metric].min(), 0)

for algorithm in algorithms:
    scores_algorithm = scores[scores["algorithm"] == algorithm]
    x_positions.loc[scores_algorithm.index]

    customdata = np.dstack(
        (
            scores_algorithm[f"{metric}_std"].values.flatten(),
            scores_algorithm["sample_size"].values.flatten(),
            [algorithm] * len(scores_algorithm.index),
        )
    )[0]
    fig.add_bar(
        x=x_positions.loc[scores_algorithm.index].values.flatten(),
        y=scores_algorithm[metric],
        error_y={"array": scores_algorithm[f"{metric}_std"], "type": "data"},
        name=ALGORITHMS_RENDERING[algorithm],
        hovertemplate=hovertemplate,
        customdata=customdata,
    )


fig.update_layout(
    yaxis={
        "title": SCORES[metric],
        "showgrid": False,
        "zeroline": False,
        "title_font": {"size": 45},
        "dtick": 1 if metric == "rmse" else 0.1,
        "tickfont_size": 20,
    },
    xaxis={"showgrid": False, "zeroline": False, "tickfont":{"size": 25}},
    # height=800,
    # width=1000,
    margin={"l": 0, "r": 0, "b": 0, "t": 0},
    legend={"orientation": "h", "y":-0.3, "font": {"size": 30}},
)

print(f"Average {SCORES[metric]} = {scores[metric].mean().round(3)} +- {scores[metric].std().round(3)}")

fig.show(config=DOWNLOAD_CONFIG)

Average R² = 0.21199999749660492 +- 0.17100000381469727


In [7]:
import plotly.graph_objs as go

sorted_dimensions = scores.index.drop_duplicates()

x_positions = pd.DataFrame(
    np.arange(5, 10 * len(sorted_dimensions) + 5, 10), index=sorted_dimensions, columns=["x_position"]
)

fig = go.Figure()
fig.update_layout(
    xaxis={
        "tickvals": np.arange(5, 10 * len(sorted_dimensions) + 5, 10),
        "ticktext": [" - ".join(elem) for elem in sorted_dimensions.values],
    }
)

algorithms = scores["algorithm"].drop_duplicates()

hovertemplate = (
    "%{x}, score: %{y:.3f} +- %{customdata[0]:.3f}, sample size: %{customdata[1]} <extra>%{customdata[2]}</extra>"
)

min_score = min(scores[metric].min(), 0)

for algorithm in algorithms:
    scores_algorithm = scores[scores["algorithm"] == algorithm]
    x_positions.loc[scores_algorithm.index]

    customdata = np.dstack(
        (
            scores_algorithm[f"{metric}_std"].values.flatten(),
            scores_algorithm["sample_size"].values.flatten(),
            [algorithm] * len(scores_algorithm.index),
        )
    )[0]
    fig.add_bar(
        x=x_positions.loc[scores_algorithm.index].values.flatten(),
        y=scores_algorithm[metric],
        error_y={"array": scores_algorithm[f"{metric}_std"], "type": "data"},
        name=ALGORITHMS_RENDERING[algorithm],
        hovertemplate=hovertemplate,
        customdata=customdata,
    )

dimensions = sorted_dimensions.to_frame()[["dimension", "subdimension", "sub_subdimension"]].reset_index(drop=True)
dimensions["position"] = fig["layout"]["xaxis"]["tickvals"]
dimensions.set_index(["dimension", "subdimension", "sub_subdimension"], inplace=True)

lines = []
annotations = []

if dimensions_selection == "custom_dimensions":
    size_dimension = 18
    size_subdimension = 15
    if metric == "rsme":
        dimension_outer_margin = min_score - 9
        dimension_inner_margin = min_score - 5
        subdimension_margin = min_score - 1
        sub_subdimension_margin = min_score - 1
    else:
        dimension_outer_margin = min_score - 0.9
        dimension_inner_margin = min_score - 0.5
        subdimension_margin = min_score - 0.1
        sub_subdimension_margin = min_score - 0.1

else:
    if selected_dimension == "all":
        size_dimension = 15
        size_subdimension = 12
        size_sub_subdimension = 11
    else:
        size_dimension = 21
        size_subdimension = 20
        size_sub_subdimension = 20

    if metric == "rmse":
        dimension_outer_margin = min_score - 17
        dimension_inner_margin = min_score - 13
        subdimension_margin = min_score - 9
        sub_subdimension_margin = min_score - 1
    else:
        dimension_outer_margin = min_score - 1.7
        dimension_inner_margin = min_score - 1.3
        subdimension_margin = min_score - 0.9
        sub_subdimension_margin = min_score - 0.1

for dimension in dimensions.index.get_level_values("dimension").drop_duplicates():
    min_position = dimensions.loc[dimension].min()
    max_position = dimensions.loc[dimension].max()

    line, annotation = add_line_and_annotation(
        dimension,
        "x",
        "y",
        min_position,
        max_position,
        dimension_inner_margin,
        dimension_outer_margin,
        90,
        size_dimension,
    )

    lines.append(line)
    annotations.append(annotation)

    for subdimension in dimensions.loc[dimension].index.get_level_values("subdimension").drop_duplicates():
        submin_position = dimensions.loc[(dimension, subdimension)].min()
        submax_position = dimensions.loc[(dimension, subdimension)].max()

        line, annotation = add_line_and_annotation(
            subdimension,
            "x",
            "y",
            submin_position,
            submax_position,
            subdimension_margin,
            dimension_inner_margin,
            90,
            size_subdimension,
        )

        lines.append(line)
        annotations.append(annotation)

        if dimensions_selection == "custom_dimensions":
            continue

        for sub_subdimension in (
            dimensions.loc[(dimension, subdimension)].index.get_level_values("sub_subdimension").drop_duplicates()
        ):
            sub_submin_position = dimensions.loc[(dimension, subdimension, sub_subdimension)].min()
            sub_submax_position = dimensions.loc[(dimension, subdimension, sub_subdimension)].max()

            line, annotation = add_line_and_annotation(
                sub_subdimension,
                "x",
                "y",
                sub_submin_position,
                sub_submax_position,
                sub_subdimension_margin,
                subdimension_margin,
                90,
                size_sub_subdimension,
            )

            lines.append(line)
            annotations.append(annotation)

# The final top/right line
line, _ = add_line_and_annotation(
    dimension,
    "x",
    "y",
    min_position,
    max_position,
    sub_subdimension_margin,
    dimension_outer_margin,
    0,
    10,
    final=True,
)

lines.append(line)

fig["layout"]["shapes"] = lines
fig["layout"]["annotations"] = annotations
fig.update_layout(xaxis={"showticklabels": False})

fig.update_layout(
    yaxis={
        "title": SCORES[metric],
        "showgrid": False,
        "zeroline": False,
        "title_font": {"size": 45},
        "dtick": 1 if metric == "rmse" else 0.1,
        "tickfont_size": 20,
    },
    xaxis={"showgrid": False, "zeroline": False},
    height=800,
    margin={"l": 0, "r": 0, "b": 0, "t": 0},
    legend={"orientation": "h", "yanchor": "bottom", "font": {"size": 30}},
)

print(f"Average {SCORES[metric]} = {scores[metric].mean().round(3)} +- {scores[metric].std().round(3)}")
fig